In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)
from torch.autograd import Variable
#from torchsummary import summary
import torch.optim as optim
import time
import matplotlib
from torch.utils.data import DataLoader
import torch.utils.data as data
import numpy as np
import pandas as pd
import py_func.Deep_learning_torch_func as DL
from torchvision.datasets import ImageFolder
from torchvision import transforms
import py_func.models_func as my_model
import py_func.dat_rb_func as drb

In [7]:
fm = drb.bin_read('./dat/bindat/1byte/fm_128_1.dat', 1,70000,128,128)

In [8]:
for i,bin in enumerate(fm):
    drb.array_to_grayjpeg(128,128,bin,str(i),'./grayimg/fm')

In [ ]:
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn


def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False,)


def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1  # 出力のチャンネル数を入力のチャンネル数の何倍に拡大するか

    def __init__(self,in_channels,channels,stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_channels, channels, stride)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(channels, channels)
        self.bn2 = nn.BatchNorm2d(channels)

        # 入力と出力のチャンネル数が異なる場合、x をダウンサンプリングする。
        if in_channels != channels * self.expansion:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(x)

        out = self.relu(out)

        return out
    
    
class Bottleneck(nn.Module):
    expansion = 4  # 出力のチャンネル数を入力のチャンネル数の何倍に拡大するか

    def __init__(self, in_channels, channels, stride=1):
        print(in_channels, out_channes)
        super().__init__()
        self.conv1 = conv1x1(in_channels, channels)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = conv3x3(channels, channels, stride)
        self.bn2 = nn.BatchNorm2d(channels)
        self.conv3 = conv1x1(channels, channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合、x をダウンサンプリングする。
        if in_channels != channels * self.expansion:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(x)

        out = self.relu(out)

        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super().__init__()

        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 重みを初期化する。
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, channels, blocks, stride):
        layers = []

        # 最初の Residual Block
        layers.append(block(self.in_channels, channels, stride))

        # 残りの Residual Block
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
    
def resnet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def resnet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def resnet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def resnet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [9]:
SX=256 #画像サイズ
N_m=70000 #データ数
N_c=60000
N_t=12966 
N_val=500

In [10]:
dp='./simdat/1byte/in_10class_256_0.2_0.1_sim_val.dat'

dp_label='./label/in_10class_256_label_val.dat'

in10_val=drb.sim_label_read(dp,SX,N_val,False,1)

label_in_val=drb.sim_label_read(dp_label,1,N_val,True,4)

In [11]:
in10_val=in10_val.reshape(N_val,1,16,16)
label_in_val=label_in_val.reshape(N_val,)

In [12]:
in10_val = torch.tensor(in10_val, dtype=torch.float32)
label_in_val=label_in_val.astype(int)
label_in_val=torch.tensor(label_in_val, dtype=torch.int64)

In [ ]:
in_set_v = torch.utils.data.TensorDataset(in10_val, label_in_val)


In [8]:
valdat=drb.sim_label_read(,256,500,False,1)

In [ ]:
vallist=DL.val_model(model)

In [5]:
model_path='./torch_pth/sim/in_256cnn_0.2_0.1_e50_b256_lr0.1_norm.pth'
model=my_model.simnet_cnn_256(10)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
DL.val_model()

In [ ]:
class Human:
    def __init__(self,name):
        self.name = name

class Greet(Human):
    def __init__(self, name, age):
        super(Greet,self).__init__(name)
        self.age = age
    
    def hello(self):
        print(self.name, 'さん こんにちは！')


In [ ]:
messi=Human(name='messi')

In [ ]:
messi.name

'messi'

In [ ]:
messigreet=Greet(name='messi',age=34)

In [ ]:
messigreet.hello()

messi さん こんにちは！


In [ ]:
messigreet.age

34

In [ ]:
messigreet.name

'messi'

In [ ]:
class Human:
    def __init__(self,name):
        self.name = name

class Greet(Human):
    def __init__(self, name, age):
        super().__init__(name)
        self.age = age
    
    def hello(self):
        print(self.name, 'さん こんにちは！')


In [ ]:
messigreet=Greet(name='messi',age=34)

In [ ]:
messigreet.hello()

messi さん こんにちは！


In [ ]:
messigreet.name

'messi'

In [ ]:
class Human:
    def __init__(self,name):
        self.name = name

class Greet(Human):
    def __init__(self, name, age):
        super().__init__(name)
        self.age = age
    
    def hello(self):
        print(self.name, 'さん こんにちは！')


In [ ]:
messigreet=Greet(name='messi',age=34)

In [ ]:
messigreet.name

'messi'

In [ ]:
psg=['neymar','embappe','messi','ramos']

In [ ]:
for i,j in enumerate(psg):
  print(i)
  print(j)

0
neymar
1
embappe
2
messi
3
ramos


In [1]:
a=[1,2,3,4,5]
b=[6,7,8,9]
a.append(b)
print(a)

[1, 2, 3, 4, 5, [6, 7, 8, 9]]


In [2]:
a=[1,2,3,4,5]
b=[6,7,8,9]
c=[]
c.append(a)
c.append(b)

In [3]:
print(c)

[[1, 2, 3, 4, 5], [6, 7, 8, 9]]


In [4]:
a=[1,2,3,4,5]
b=[6,7,8,9]
c=[]
c.append(a,b)

TypeError: ignored